# Retrogate Analysis

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-symmetry.ipynb

import pymongo
localMongo = pymongo.MongoClient("mongodb://localhost:27017/")
retroDb = localMongo["retro-database"]
retroCol = retroDb["retro-collection"]

In [ ]:
startBoard = ((' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
boards = set()

def generateBoards(board=startBoard):
    global boards
    count = 0
    for a1 in range(0,3):
        for a2 in range(0, 8):
            count += 1
            print('Progress: ' + str(count) + '/24')
            for b1 in range(0,3):
                for b2 in range(0, 8):
                    for c1 in range(0,3):
                        for c2 in range (0, 8):
                            for d1 in range(0,3):
                                for d2 in range(0, 8):
                                    for e1 in range(0,3):
                                        for e2 in range(0,8):
                                            for f1 in range(0,3):
                                                for f2 in range(0,8):
                                                    board1 = place(board, (a1, a2), 'w')
                                                    board2 = place(board1, (b1, b2), 'b')
                                                    board3 = place(board2, (c1, c2), 'w')
                                                    board4 = place(board3, (d1, d2), 'b')
                                                    board5 = place(board4, (e1, e2), 'w')
                                                    board6 = place(board5, (f1, f2), 'b')
                                                    white = countStones(('', board6), 'w')
                                                    black = countStones(('', board6), 'b')
                                                    if (white == 3 and black == 3):
                                                        boards.add(board6)

In [ ]:
def getUniqueStateForSymmetry(state):
    symmetries = findSymmetries(state)
    
    state = None
    maxWeight = float('-inf') 
    for s in symmetries:
        weight = hash(s)
        if weight > maxWeight:
            state = s
            maxWeight = weight
    return state

In [ ]:
def getSymmetryUnique(boards):
    uniqueStates = set()
    stones = (0, 0)
    for board in boards:
        state = tuple([stones, board])
        uniqueStates.add(getUniqueStateForSymmetry(state))
    return uniqueStates

In [ ]:
def fillDb(states):
    fullStep = []
    stepCount = 0
    
    stepCount += 1
    
    fullStates = states.copy()
    halfStates = states.copy()
    _states = fullStates.copy()

    for state in _states:
        _, board = state
        if (findPossibleMills(board, 'w')) == set():
            continue;
        _nextStates = nextStates(state, 'w')
        for ns in _nextStates:
            if (finished(ns, 'w') and utility(ns, 'w') == 1):
                entry = { "state": state, "nextState": ns, "steps": stepCount }
                retroCol.insert_one(entry)
                fullStep.append(state)
                fullStates.remove(state)
                break
    
    while len(states) > 0:
        enrichedFullStep = set()
        for s in fullStep:
            enrichedFullStep |= findSymmetries(s)
        
        _states = halfStates.copy()
        halfStep = []
        stepCount += 1       
        
        for state in _states:
            _nextStates = nextStates(state, 'b')
            if (_nextStates.issubset(enrichedFullStep)):
                halfStep.append(state)
                halfStates.remove(state)
                    
        if len(halfStep) == 0:
            break
        
        enrichedHalfStep = set()
        for s in halfStep:
            enrichedHalfStep |= findSymmetries(s)
        
        _states = fullStates.copy()
        
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'w')
            for ns in _nextStates:
                if (ns in enrichedHalfStep):
                    entry = { "state": state, "nextState": ns, "steps": stepCount }
                    retroCol.insert_one(entry)
                    fullStep.append(state)
                    fullStates.remove(state)
                    break

In [ ]:
generateBoards()
global boards
states = getSymmetryUnique(boards)
fillDb(states)